# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117537e+02     1.656400e+00
 * time: 0.08872580528259277
     1     1.075710e+01     9.504320e-01
 * time: 0.9799149036407471
     2    -1.211842e+01     1.091357e+00
 * time: 1.1040408611297607
     3    -3.418029e+01     8.510157e-01
 * time: 1.2557408809661865
     4    -4.765542e+01     6.710867e-01
 * time: 1.4032578468322754
     5    -5.714002e+01     1.875801e-01
 * time: 1.5433988571166992
     6    -5.991139e+01     1.538177e-01
 * time: 1.654331922531128
     7    -6.098209e+01     6.158659e-02
 * time: 1.758227825164795
     8    -6.141048e+01     4.685478e-02
 * time: 1.8697149753570557
     9    -6.165005e+01     4.852932e-02
 * time: 1.9809727668762207
    10    -6.183024e+01     3.266474e-02
 * time: 2.0851528644561768
    11    -6.196543e+01     2.254518e-02
 * time: 2.1951968669891357
    12    -6.204963e+01     1.636796e-02
 * time: 2.297775983810425
    13    -6.209515e+01     1.561382e-02
 * time: 2.408371925354

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522619
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485377 
    Xc                  -19.3336823

    total               -62.261666459829
